In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots


In [62]:
# Read data from the gov's website excel link and save it to df_i as import and df_e as export (July 2022, update: 12/9)

df_i = pd.read_excel("https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1103220/OTS_IMP_2207.xlsx",
                     header = 2)
df_i = df_i.rename(columns={"Unnamed: 0": "Number", "July 2022": "Cost", "Goods Description": "Goods", "Year to Date 2022": "YTD_Cost"})
df_i.drop(index=df_i.index[-1],axis=0,inplace=True)

df_e = pd.read_excel("https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1103222/OTS_EXP_2207.xlsx",
                     header = 2)
df_e = df_e.rename(columns={"Unnamed: 0": "Number", "July 2022": "Cost", "Goods Description": "Goods", "Year to Date 2022": "YTD_Cost"})
df_e.drop(index=df_e.index[-1],axis=0,inplace=True)

In [85]:
# Tidy up the data by renaming and merge the two data into df_t

df_t = (pd.merge(df_i, df_e, how = 'inner', left_on = ['Goods'], right_on = ['Goods'])).drop(columns = ['Chap', 'Number'])
df_t = df_t.rename(columns={"Cost_x": "Cost_i", "Cost_y": "Cost_e", "Year to Date  2022_x": "YTD_i", "Year to Date  2022_y": "YTD_e"})
df_t['Cost_e'] = df_t['Cost_e'].apply(lambda x: x*-1000)
df_t['Cost_i'] = df_t['Cost_i'].apply(lambda x: x*1000)
df_t['YTD_i'] = df_t['YTD_i'].apply(lambda x: x*1000)
df_t['YTD_e'] = df_t['YTD_e'].apply(lambda x: x*-1000)

total = df_t.Cost_i + df_t.Cost_e
total_t = df_t.YTD_i + df_t.YTD_e
df_t['Cost_total'] = total.tolist()
df_t['YTD_total'] = total_t.tolist()
df_t

,Goods,Cost_i,YTD_i,Cost_e,YTD_e,Cost_total,YTD_total
0,Live animals,84649000,477708000,-52812000,-278125000,31837000,199583000
1,Meat and edible meat offal,337309000,2382993000,-117214000,-784888000,220095000,1598105000
2,"Fish and crustaceans, molluscs and other aquat...",33520000,181700000,-100036000,-616734000,-66516000,-435034000
3,Dairy produce; birds' eggs; natural honey; edi...,319386000,1798458000,-124170000,-855025000,195216000,943433000
4,Products of animal origin not elsewhere specif...,9851000,74769000,-12710000,-81075000,-2859000,-6306000
...,...,...,...,...,...,...,...
92,Furniture; medical and surgical furniture; bed...,453888000,3066753000,-130928000,-876346000,322960000,2190407000
93,"Toys, games and sports requisites; parts and a...",112047000,795951000,-63949000,-506281000,48098000,289670000
94,Miscellaneous manufactured articles,94018000,624225000,-18753000,-129519000,75265000,494706000
95,"Works of art, collectors' pieces and antiques",72872000,420464000,-66912000,-686639000,5960000,-266175000


In [95]:
# Visualising data and later export

# fig = px.bar(df_t, x="Goods", y=["Cost_i", "Cost_e", "total"], title="Import x Export", width=1300, height = 2300)
# fig.show()

fig = px.bar(df_t, x=df_t.Goods[:5], y=[df_t.YTD_i[:5], df_t.YTD_e[:5]], 
             title="Import x Export",
             labels={"Cost_i": "Cost_i", "Cost_e": "Cost_e"},
#              name="go",
             width=1000, 
             height = 1000,
             text_auto='.2s')
# fig.update_layout(legend_traceorder="reversed")

newnames = {'wide_variable_0':'Import', 'wide_variable_1': 'Export'}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.show()

In [92]:
fig.write_image(r"C:\Users\Rig 1\Desktop\Code\Python Project\Results\bar.png", scale=5)

In [87]:
fig_i = px.bar(df_t, x=df_t.Cost_i[:5], y=df_t.Goods[:5])
fig_i.update_layout(
    autosize=False,
    width=1300,
    height=600
)

In [7]:
fig.write_image(r"C:\Users\Rig 1\Desktop\Code\Python Project\Results\fig4.jpeg", scale=3)

In [8]:
fig_e = px.bar(df_t, x=df_t.Cost_e[:5], y=df_t.Goods[:5], color_discrete_sequence=['red'])
fig_e.update_layout(
    autosize=False,
    width=2500,
    height=1800
)

In [9]:
fig3 = go.Figure(data=fig_i.data + fig_e.data)
fig3.update_layout(
    autosize=False,
    width=1000,
    height=600
#     margin=dict(
#         l=50,
#         r=50,
#         b=100,
#         t=100,
#         pad=4
#     ),
#     paper_bgcolor="LightSteelBlue",
)
fig3.show()

In [10]:
fig3.write_image(r"C:\Users\Rig 1\Desktop\Code\Python Project\Results\bar.jpeg", scale=2)